<a href="https://colab.research.google.com/github/mgs06380/SNA_Research/blob/main/1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import numpy as np
import pandas as pd
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import glob
from tqdm import tqdm
import joblib

# 구글 드라이브 마운트
drive.mount('/content/drive')

# tqdm 스타일 설정
tqdm.pandas()

# 파일 경로 설정
base_path = '/content/drive/MyDrive/DSRC'
nodes_path = f"{base_path}/nodes.csv"
links_path_pattern = f"{base_path}/Links/Links_04*.csv"
entry_exit_path_pattern = f"{base_path}/진출진입/entering_04*.csv"

# 노드 데이터 불러오기
print("노드 데이터 로드 중...")
nodes_df = pd.read_csv(nodes_path, encoding='utf-8')

# Links 데이터 불러오기 (여러 파일을 병합)
print("Links 데이터 로드 중...")
links_files = glob.glob(links_path_pattern)
links_df = pd.concat((pd.read_csv(f, encoding='utf-8') for f in tqdm(links_files)), ignore_index=True)

# Entering 데이터 불러오기 (여러 파일을 병합)
print("Entering 데이터 로드 중...")
entry_exit_files = glob.glob(entry_exit_path_pattern)
entry_exit_df = pd.concat((pd.read_csv(f, encoding='utf-8') for f in tqdm(entry_exit_files)), ignore_index=True)

# 각 데이터프레임의 첫 몇 행을 확인
print("Nodes DataFrame:")
display(nodes_df.head())

print("\nLinks DataFrame:")
display(links_df.head())

print("\nEntering DataFrame:")
display(entry_exit_df.head())

# 컬럼 이름 확인
print("Nodes DataFrame Columns:", nodes_df.columns.tolist())
print("Links DataFrame Columns:", links_df.columns.tolist())
print("Entering DataFrame Columns:", entry_exit_df.columns.tolist())

# 차량 경로 시퀀스 생성
print("차량 경로 시퀀스 생성 중...")
vehicle_paths = defaultdict(list)
for _, row in tqdm(links_df.iterrows(), total=links_df.shape[0]):
    vehicle_paths[row['차량ID']].append(row['Source'])
    vehicle_paths[row['차량ID']].append(row['Target'])

for vehicle_id in vehicle_paths:
    vehicle_paths[vehicle_id] = list(dict.fromkeys(vehicle_paths[vehicle_id]))

sequence_data = []
for vehicle_id, path in vehicle_paths.items():
    for i in range(len(path) - 1):
        sequence_data.append([vehicle_id, path[i], path[i + 1]])

sequence_df = pd.DataFrame(sequence_data, columns=['차량ID', 'Source', 'Target'])
print("\nSequence DataFrame:")
display(sequence_df.head())

# 샘플링을 60%로 적용
sequence_df = sequence_df.sample(frac=0.6, random_state=42)

# 노드 ID에 라벨 인코딩 적용
print("노드 ID 라벨 인코딩 중...")
label_encoder = LabelEncoder()
all_nodes = pd.concat([sequence_df['Source'], sequence_df['Target']])
label_encoder.fit(all_nodes)

# 데이터프레임을 인코딩된 노드로 업데이트
sequence_df['Source'] = label_encoder.transform(sequence_df['Source'])
sequence_df['Target'] = label_encoder.transform(sequence_df['Target'])

# 시퀀스와 레이블 생성
print("시퀀스와 레이블 생성 중...")
sequences = []
next_nodes = []

for vehicle_id, group in sequence_df.groupby('차량ID'):
    seq = group['Source'].tolist()
    target = group['Target'].tolist()
    for i in range(1, len(seq)):
        sequences.append(seq[:i])
        next_nodes.append(target[i-1])

# 시퀀스 패딩 및 레이블 변환을 배치 단위로 처리
print("시퀀스 패딩 및 레이블 변환 배치 처리 중...")
batch_size = 100000  # 배치 크기 설정
num_batches = int(np.ceil(len(sequences) / batch_size))
max_sequence_length = max(len(seq) for seq in sequences)

sequences_padded_list = []
next_nodes_categorical_list = []

for i in range(num_batches):
    batch_sequences = sequences[i * batch_size: (i + 1) * batch_size]
    batch_next_nodes = next_nodes[i * batch_size: (i + 1) * batch_size]

    batch_sequences_padded = pad_sequences(batch_sequences, maxlen=max_sequence_length, padding='pre')
    batch_next_nodes_categorical = to_categorical(batch_next_nodes, num_classes=len(label_encoder.classes_)).astype(np.float32)

    sequences_padded_list.append(batch_sequences_padded)
    next_nodes_categorical_list.append(batch_next_nodes_categorical)

sequences_padded = np.concatenate(sequences_padded_list, axis=0)
next_nodes_categorical = np.concatenate(next_nodes_categorical_list, axis=0)

# Entering 데이터도 샘플링을 60%로 적용
entry_exit_df = entry_exit_df.sample(frac=0.6, random_state=42)

# Entering 데이터도 인코딩
print("Entering 데이터 인코딩 중...")
entry_exit_df['RSE_ID'] = label_encoder.transform(entry_exit_df['RSE_ID'])

# 각 시퀀스에 해당하는 진입/진출 데이터를 반복하여 맞추기
entry_exit_data_dict = entry_exit_df.drop_duplicates(subset=['RSE_ID']).set_index('RSE_ID')[['EnteringCount', 'ExitingCount']].to_dict('index')
entry_exit_data = [entry_exit_data_dict[node] if node in entry_exit_data_dict else {'EnteringCount': 0, 'ExitingCount': 0} for node in sequence_df['Source']]
entry_exit_data = pd.DataFrame(entry_exit_data).astype(np.float32)

# 데이터 저장
print("데이터 저장 중...")
joblib.dump((sequences_padded, entry_exit_data, next_nodes_categorical, max_sequence_length, label_encoder), '/content/drive/MyDrive/preprocessed_data.pkl')
print("데이터 저장 완료")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
노드 데이터 로드 중...
Links 데이터 로드 중...


100%|██████████| 30/30 [01:14<00:00,  2.48s/it]


Entering 데이터 로드 중...


100%|██████████| 30/30 [00:15<00:00,  1.88it/s]

Nodes DataFrame:


...


Entering DataFrame:


,Index,위도값,경도값,RSE_ID,ExitingCount,EnteringCount,Address
0,0,37.16678,127.09193,2100401013808,180514,183103,대한민국 경기도 화성시 송동 425-9
1,1,37.09439,127.12184,2100401013713,152954,153573,대한민국 경기도 용인시 처인구 남사읍 진목리 949-383
2,2,37.12583,127.09694,2100401013760,139702,138832,대한민국 경기도 평택시 진위면 동천리 602-2
3,3,37.27944,127.10353,2100401013936,138791,130950,대한민국 경기도 용인시 기흥구 신갈동 384-10
4,4,37.15923,127.08844,2100401013802,136701,139482,대한민국 경기도 오산시 부산동 157-3


Nodes DataFrame Columns: ['Node']
Links DataFrame Columns: ['차량ID', 'Source', 'Target']
Entering DataFrame Columns: ['Index', '위도값', '경도값', 'RSE_ID', 'ExitingCount', 'EnteringCount', 'Address']
차량 경로 시퀀스 생성 중...


  3%|▎         | 4085061/160669202 [02:40<1:42:49, 25379.14it/s]


KeyboardInterrupt: 

In [ ]:
# 두 번째 셀: 모델 학습 및 예측

import joblib
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Concatenate, Input
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from tqdm import tqdm

# 데이터 로드
print("데이터 로드 중...")
sequences_padded, entry_exit_data, next_nodes_categorical, max_sequence_length, label_encoder = joblib.load('/content/drive/MyDrive/preprocessed_data.pkl')

# 데이터 확인
print("sequences_padded shape:", sequences_padded.shape)
print("entry_exit_data shape:", entry_exit_data.shape)
print("next_nodes_categorical shape:", next_nodes_categorical.shape)
print("max_sequence_length:", max_sequence_length)
print("label_encoder classes:", label_encoder.classes_)

# 데이터 크기 확인
print("sequences_padded samples:", len(sequences_padded))
print("entry_exit_data samples:", len(entry_exit_data))

# 데이터 크기가 일치하지 않으면 데이터 크기를 맞추기 위해 잘라내기
min_samples = min(len(sequences_padded), len(entry_exit_data))
sequences_padded = sequences_padded[:min_samples]
entry_exit_data = entry_exit_data.iloc[:min_samples]

# LSTM 모델 생성
print("LSTM 모델 생성 중...")
embedding_dim = 128
lstm_units = 64

node_input = Input(shape=(max_sequence_length,), name='node_input')
entry_exit_input = Input(shape=(entry_exit_data.shape[1],), name='entry_exit_input')

embedding_layer = Embedding(input_dim=len(label_encoder.classes_), output_dim=embedding_dim)(node_input)
lstm_layer = LSTM(units=lstm_units, return_sequences=False)(embedding_layer)

merged_layer = Concatenate()([lstm_layer, entry_exit_input])
output_layer = Dense(units=len(label_encoder.classes_), activation='softmax')(merged_layer)

model = Model(inputs=[node_input, entry_exit_input], outputs=output_layer)

# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 데이터 배치 크기 설정
batch_size = 2048  # 배치 크기 늘리기
num_epochs = 10  # 에폭 수 늘리기

# 학습 데이터와 검증 데이터로 나누기
train_indices, val_indices = train_test_split(np.arange(len(sequences_padded)), test_size=0.2, random_state=42)
train_sequences = sequences_padded[train_indices]
train_entry_exit = entry_exit_data.iloc[train_indices]
train_labels = next_nodes_categorical[train_indices]
val_sequences = sequences_padded[val_indices]
val_entry_exit = entry_exit_data.iloc[val_indices]
val_labels = next_nodes_categorical[val_indices]

# 모델 학습 및 학습 기록 저장
print("모델 학습 중...")
history = model.fit(
    [train_sequences, train_entry_exit], train_labels,
    validation_data=([val_sequences, val_entry_exit], val_labels),
    epochs=num_epochs,
    batch_size=batch_size,
    verbose=1
)

# 학습 기록 시각화
plt.figure(figsize=(12, 4))

# 손실 그래프
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

# 정확도 그래프
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.show()

# 예측 수행
print("예측 수행 중...")
predictions = model.predict([sequences_padded, entry_exit_data])

# 각 시퀀스에 대해 가장 높은 확률을 가진 노드 인덱스 가져오기
predicted_nodes = predictions.argmax(axis=1)

# 예측된 노드 인덱스를 원래 노드 ID로 매핑
predicted_node_ids = label_encoder.inverse_transform(predicted_nodes)

# 각 예측된 노드의 발생 빈도 계산
node_counts = pd.Series(predicted_node_ids).value_counts()

# 가장 혼잡한 노드 출력
congested_nodes = node_counts.head(10)
print("Top 10 most congested nodes:")
print(congested_nodes)

# 예측된 혼잡 노드 정보를 추출
congested_nodes_df = pd.DataFrame({
    'Node': congested_nodes.index,
    'PredictedCount': congested_nodes.values
})

# 기존 데이터프레임에서 주소 정보 가져오기
entry_exit_files = glob.glob(entry_exit_path_pattern)
entry_exit_df = pd.concat((pd.read_csv(f, encoding='utf-8') for f in entry_exit_files), ignore_index=True)
address_mapping = entry_exit_df[['RSE_ID', 'Address']].drop_duplicates().set_index('RSE_ID')

# 주소 정보를 예측된 혼잡 노드 정보와 병합
congested_nodes_df = congested_nodes_df.join(address_mapping, on='Node')

# 결과를 깔끔한 표 형식으로 출력
print("예측된 혼잡 노드 정보:")
display(congested_nodes_df[['Node', 'PredictedCount', 'Address']])
